# Inteprolating missing transactional data
## TODOs:
- Make it so there's same amount of staff weekly?

### Data used
- Mendeley data
    - Was augmented with a possible part-time datapoint
    - First x datapoints used


In [173]:
from scipy.interpolate import interp1d
import numpy as np
import pandas as pd
import csv
import math
import matplotlib.pyplot as plt

In [174]:
fileName = "_FullTransactions_xMonth_Data_File.csv"

In [175]:
file_to_read = '.././Data/' + fileName
file_to_write = '.././Data/_WithStaff_' + fileName
df = pd.read_csv(file_to_read) 


In [176]:
df['transaction_count'][0]

11

In [177]:
# Find the day with the lowest transaction_count in both years
lowest_transaction_count =  10000
lowest_transaction_count_day =  df['Timestamp'][0]

newDf = df.copy()


for i in range(0,len(newDf)-10,9):
    temp_lowest = 0

    if(newDf['transaction_count'][i:i+9].all() != 0):

        for a in range(i,i+9):
            #print("Adding: ", newDf['Timestamp'][a], newDf['transaction_count'][a])
            temp_lowest += newDf['transaction_count'][a]

        if(temp_lowest < lowest_transaction_count):
            lowest_transaction_count = temp_lowest
            lowest_transaction_count_day = newDf['Timestamp'][i]


lowest_transaction_count_day, lowest_transaction_count

('04/04/2022 10:00', 90)

In [178]:
newDf[2900:2910]

,Timestamp,subtotal,transaction_count,rain,temperature,holiday,oil_price,workforce_type_1,workforce_type_2,workforce_type_3,workforce_type_4,Unnamed: 11
2900,04/04/2022 12:00,1486.78,22,0.0,13.7,False,108.15,1.0,2.0,3.0,1.0,NaN
2901,04/04/2022 13:00,1496.58,21,0.0,13.1,False,108.15,1.0,2.0,3.0,1.0,NaN
2902,04/04/2022 14:00,169.90,3,0.0,13.3,False,108.15,1.0,2.0,3.0,1.0,NaN
2903,04/04/2022 15:00,174.08,2,0.0,12.9,False,108.15,1.0,2.0,3.0,1.0,NaN
2904,04/04/2022 16:00,556.50,12,0.0,12.3,False,108.15,1.0,2.0,3.0,1.0,NaN
2905,04/04/2022 17:00,855.43,10,0.0,12.1,False,108.15,1.0,2.0,3.0,0.0,NaN
2906,04/04/2022 18:00,100.91,2,0.0,11.4,False,108.15,1.0,2.0,3.0,0.0,NaN
2907,05/04/2022 10:00,804.60,12,0.0,10.5,False,106.60,NaN,NaN,NaN,NaN,NaN
2908,05/04/2022 11:00,1709.63,24,0.0,11.6,False,106.60,NaN,NaN,NaN,NaN,NaN
2909,05/04/2022 12:00,1538.06,24,0.0,12.2,False,106.60,NaN,NaN,NaN,NaN,NaN


In [179]:
baselineStaff_1 = newDf['workforce_type_1'][2898:2907].reset_index(drop=True)
baselineStaff_2 = newDf['workforce_type_2'][2898:2907].reset_index(drop=True)
baselineStaff_3 = newDf['workforce_type_3'][2898:2907].reset_index(drop=True)
baselineStaff_4 = newDf['workforce_type_4'][2898:2907].reset_index(drop=True)
baselineStaff_4

0    0.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    0.0
8    0.0
Name: workforce_type_4, dtype: float64

In [183]:
from datetime import datetime, timedelta

DATE_FORMAT = "%d/%m/%Y %H:%M"

newDf['Timestamp'] = newDf['Timestamp'].apply(lambda x: datetime.strptime(x, DATE_FORMAT))

newDf

,Timestamp,subtotal,transaction_count,rain,temperature,holiday,oil_price,workforce_type_1,workforce_type_2,workforce_type_3,workforce_type_4,Unnamed: 11
0,2021-05-17 10:00:00,428.03,11,0.0,12.7,False,69.62,NaN,2.0,3.0,NaN,NaN
1,2021-05-17 11:00:00,324.78,15,0.0,13.4,False,69.62,NaN,NaN,NaN,NaN,NaN
2,2021-05-17 12:00:00,279.75,12,0.0,13.6,False,69.62,NaN,NaN,NaN,NaN,NaN
3,2021-05-17 13:00:00,1630.59,29,0.0,14.1,False,69.62,NaN,NaN,NaN,NaN,NaN
4,2021-05-17 14:00:00,1166.10,18,0.0,15.9,False,69.62,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5071,2022-12-01 14:00:00,2427.05,58,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
5072,2022-12-01 15:00:00,2530.32,54,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
5073,2022-12-01 16:00:00,3383.75,70,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
5074,2022-12-01 17:00:00,2424.71,58,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN


In [190]:
#baselineStaff_2.add(1)
newDf['Timestamp'][0].month

5

In [202]:
# Go through all days, seeing how much of a transaction_count is returned compared to the lowest. Add that amount to the staff count if above 1.5
for i in range(0,len(newDf)-10,9):
    daily_trasaction_count = 0

    for a in range(i,i+9):
        daily_trasaction_count += newDf['transaction_count'][a]

    #xy iterates only through the baseline staff
    xy = 0
    for a in range(i,i+9):
        transMultiple = math.floor((round(daily_trasaction_count / lowest_transaction_count) - 1)/2) # Dividing so it could be spread across staffs

        # Upperbound
        if(transMultiple > 5):
            transMultiple = 5

        newDf.at[a,'workforce_type_1'] = baselineStaff_1[xy]

        newDf.at[a,'workforce_type_2'] = baselineStaff_2.add(transMultiple)[xy]
        newDf.at[a,'workforce_type_3'] = baselineStaff_3.add(transMultiple)[xy]

        # If it's Christmas, add an extra part timer too
        if(newDf['Timestamp'][i].month == 12 and newDf['workforce_type_4'][xy] != 0):
            newDf.at[a,'workforce_type_4'] = baselineStaff_4.add(1)[xy]
        else:
            # Monday and Tuesday are least busiest days, don't need part-timers then
            mondayOrTuesday = newDf['Timestamp'][i].dayofweek == 1 or newDf['Timestamp'][i].dayofweek == 2
            if(not mondayOrTuesday):
                newDf.at[a,'workforce_type_4'] = baselineStaff_4[xy]
            else:
                newDf.at[a,'workforce_type_4'] = 0

        xy += 1

#newDf.head(10)
#newDf[1000:1200]

In [203]:
# Add to csv
newDf.to_csv(file_to_write)